# Pydantic Tutorial

- Here in the function definition we don't fix any datatype, so any data can be passed which is not desired as it breaks the consistency.

- Type hinting can be used but it just shows a suggestion, it doesn't prevent or throws any error.

In [1]:
def insert_patient_data(name, age):
    ## We are mimicking the database insert operation
    print(name)
    print(age)
    print("Inserted into database")
    

In [2]:
insert_patient_data('Tufan', 'Twenty three')


Tufan
Twenty three
Inserted into database


### What we can do is we can do a manual check like:


In [3]:

def insert_patient_data(name, age):
    ## We are mimicking the database insert operation
    if type(name) == str and type(age) == int:
        print(name)
        print(age)
        print("Inserted into database")
    else:
        raise TypeError("Incorrect data type") 


In [4]:
# This throws an error
insert_patient_data('Tufan', 'Twenty three')


TypeError: Incorrect data type

In [5]:
# this is allowed 
insert_patient_data("Tufan", 23)

Tufan
23
Inserted into database


### This works but this is not scalable. Also validation is not done here.



## Using Pydantic library

In [6]:
from pydantic import BaseModel
from typing import List, Dict, Optional

In [7]:
## Making class for validation
class Patient(BaseModel):
    ## Type validation
    name : str
    age : int
    weight : float
    married : bool = False # By default False
    allergies : Optional[List[str]] = None # making this field optional, list of string
    contact_details : Dict[str,str]
    

In [8]:
patient_info = {"name": "Tufan", 
                "age": 23, 
                "weight":61.5, 
                # "married":True, 
                "allergies": ['pollen','dust'], 
                "contact_details":{"email":"abc@gmail.com","phone":"235468954"}}

patient1 = Patient(**patient_info)



In [9]:
def insert_patient_data(patient : Patient):
    print(patient.name)
    print(patient.age)
    print(patient.weight)
    print(patient.married)
    print(patient.allergies)
    print(patient.contact_details)
    print("Inserted")
    
    
def update_patient_data(patient : Patient):
    print(patient.name)
    print(patient.age)
    print(patient.weight)
    print("Updated")
    

In [10]:
insert_patient_data(patient1)


Tufan
23
61.5
False
['pollen', 'dust']
{'email': 'abc@gmail.com', 'phone': '235468954'}
Inserted


In [11]:
update_patient_data(patient1)

Tufan
23
61.5
Updated


### Using custom type validator in pydantic

In [21]:
from pydantic import BaseModel, EmailStr, AnyUrl
from typing import List, Dict, Optional

In [22]:
class Patient(BaseModel):
    name : str
    email : EmailStr ## checking valid email or not
    linkedin_url : AnyUrl
    age : int
    weight : float
    married : bool = False # By default False
    allergies : Optional[List[str]] = None
    contact_details : Dict[str,str]
    

In [28]:
patient_info = {"name": "Tufan", 
                "email":"hulululu@gmail.com", # correct
                #"email":"hulululugmail.com", # will throw error as email is not in correct format
                "linkedin_url":"https://linkedin.com/tk", #valid url
                # "linkedin_url":"linkedin.com/tk", #invalid URL
                "age": 23, 
                "weight":61.5, 
                "married":True, 
                "allergies": ['pollen','dust'], 
                "contact_details":{"phone":"235468954"}}

patient1 = Patient(**patient_info)



In [29]:
def insert_patient_data(patient : Patient):
    print(patient.name)
    print(patient.email)
    print(patient.linkedin_url)
    print(patient.age)
    print(patient.weight)
    print(patient.married)
    print(patient.allergies)
    print(patient.contact_details)
    print("Inserted")
    

In [30]:
insert_patient_data(patient1)

Tufan
hulululu@gmail.com
https://linkedin.com/tk
23
61.5
True
['pollen', 'dust']
{'phone': '235468954'}
Inserted


### Data Validation

In [38]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
# Field is used to do custom data type check, also allows
# to attach meta data about a field(variable)

from typing import List, Dict, Optional, Annotated

In [43]:
class Patient(BaseModel):
    name : Annotated[str, Field(max_length = 50, title = 'Name of the patient', description='Give the name of the patient in less than 50 characters', examples=['Tufan', 'Shankar'])]
    email : EmailStr ## checking valid email or not
    linkedin_url : AnyUrl
    age : int = Field (gt = 0, lt = 120) #>0,<120
    weight : Annotated[float, Field(gt = 0, strict = True)] # strict = True ensures there is no automatic type conversion, if type not matches with the defined, it throws an error
    married : Annotated[bool, Field(default=None, description= 'Is the patient married or not')]
    allergies : Annotated[Optional[List[str]], Field(default = None, max_length=5)]
    contact_details : Dict[str,str]
    

In [44]:
patient_info = {"name": "Tufan", 
                "email":"hulululu@gmail.com", # correct
                #"email":"hulululugmail.com", # will throw error as email is not in correct format
                "linkedin_url":"https://linkedin.com/tk", #valid url
                # "linkedin_url":"linkedin.com/tk", #invalid URL
                "age": 23, 
                "weight":61.5, 
                "married":True, 
                "allergies": ['pollen','dust'], 
                "contact_details":{"phone":"235468954"}}

patient1 = Patient(**patient_info)

